# Working with Landsat Data in the Amazon Web Services (AWS) Cloud

This tutorial demonstrates how users discover Landsat Data stored within the AWS Cloud environment by searching SAT-API. Landsat data stored in the AWS Cloud is located within U.S. West (Oregon) us-west-2 region in a requester pays Simple Storage Services (S3) bucket. Users interested in utilizing direct access to Landsat data stored in S3 are encouraged to visit the Requester Pays Documentation Page (https://docs.aws.amazon.com/AmazonS3/latest/dev/RequesterPaysBuckets.html) for information concerning potential egress costs to accessing Landsat data with this method. 

<b>AWS Bucket Name:</b> usgs-landsat (Requester Pays) <br />
<b>AWS Region:</b> US West (Oregon) us-west-2<br />

An AWS account is required before undertaking this tutorial. Please see the AWS Account website to establish an account (https://aws.amazon.com/account/).  <br />


### Search USGS Sat-API

In [ ]:
import os
import sys
import json
import requests
import datetime
import itertools
import pandas as pd

sat_api_url = "https://landsatlook.usgs.gov/sat-api"

#### Request SAT-API Endpoint

In [ ]:
satAPI = requests.post(sat_api_url)
if satAPI.status_code == 200:
    print('Sat-API is Available')
else:
    print('Sat-API is not Available')
    sys.exit()

In [ ]:
#Lets see what was returned by the endpont
print(json.dumps(satAPI.json(), indent=2))

In [ ]:
#Lets look at the collections
satAPICollections = requests.post('https://landsatlook.usgs.gov/sat-api/collections')
print(json.dumps(satAPICollections.json(), indent=2))

In [ ]:
#lets look at the landsat-c2l2-sr collection
c2L2SR = requests.post('https://landsatlook.usgs.gov/sat-api/collections/landsat-c2l2-sr')
print(json.dumps(c2L2SR.json(), indent=2))

In [ ]:
#lets pick a random product in the landsat-c2l2-sr Collection and look at the STAC Metadata
srProduct = requests.post ('https://landsatlook.usgs.gov/sat-api/collections/landsat-c2l2-sr/items')
print(json.dumps(srProduct.json()['features'][1]['geometry'], indent=2))

### Setup a Sat-API Query

In [ ]:
# This will handle pagination
def fetch_sat_api(query):
    headers = {
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip",
        "Accept": "application/geo+json",
    }

    url = f"{sat_api_url}/stac/search"
    data = requests.post(url, headers=headers, json=query).json()
    error = data.get("message", "")
    if error:
        raise Exception(f"SAT-API failed and returned: {error}")

    meta = data.get("meta", {})
    if not meta.get("found"):
        return []
    print(meta)

    features = data["features"]
    if data["links"]:
        curr_page = int(meta["page"])
        query["page"] = curr_page + 1
        query["limit"] = meta["limit"]

        features = list(itertools.chain(features, fetch_sat_api(query)))

    return features

In [ ]:
#set up query
#Find Scenes for Path 30, Row 28 in Collection 2 L2_SR Collection acquired between 1-January 1982 and 30-December 2019
#where scene cloud cover is between 0 and 30
min_cloud = 0
max_cloud = 30

date_min="1982-01-01"
date_max="2019-12-30"

start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "time": f"{start}/{end}",
    "query": {
        "landsat:wrs_path": {"in": ["30","030"]}, #landsat Path and Row are not 0 padded so that is a bug we will need to fix
        "landsat:wrs_row": {"in": ["28","028"]}, #landsat Path and Row are not 0 padded so that is a bug we will need to fix
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud},
        "collection":{"eq": "landsat-c2l2-sr"}
    },
    "limit": 500 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
}

features = fetch_sat_api(query)

In [ ]:
#Setup Table to show results
df = pd.DataFrame(features)
#Add Acqusition Dates
dates = []
for item in df['properties']:
    dates.append(item['datetime'])
#Add Product IDs
    ids = []
for item in df['id']:
    ids.append(item)
#Add Green Bands
#L8 has different band numbering for Green this accounts for that. Green = B3 L8 TM, ETM = B2
greens = []
for item in df['assets']:
    if 'Green' in item['SR_B3.TIF']['title']: 
        greens.append(item['SR_B3.TIF']['href'])
    else:
        greens.append(item['SR_B2.TIF']['href'])
#Add SWIR Bands 
#l8 SWIR Band = B6, L7 = B7, TM = B5 this accounts for that.
        swirs = []
for item in df['assets']:
    try:
        if 'Short-wave' in item['SR_B6.TIF']['title']:
            swirs.append(item['SR_B6.TIF']['href'])
    except:
        if 'Short-wave' in item['SR_B5.TIF']['title']:
            swirs.append(item['SR_B5.TIF']['href'])
#Combine into one table
ids = pd.DataFrame(ids, columns = ['Product ID'])
dates = pd.DataFrame(dates, columns = ['Date'])
greens = pd.DataFrame(greens, columns = ['Green'])
swirs = pd.DataFrame(swirs, columns = ['Swirs'])
df = pd.concat([ids, dates, greens, swirs], axis = 1, sort=False)

In [ ]:
#Display DF, object links are the cloudfront links which will re-direct to ERS to login need to convert these to S3 object links
df

In [ ]:
#remove everything forward of /collection from the cloudfront object URL
greens = pd.DataFrame(greens['Green'].str[33:])
swirs = pd.DataFrame(swirs['Swirs'].str[33:])
#append s3://usgs-landsat to the front of the URLs
greens = pd.DataFrame("s3://usgs-landsat"+greens['Green'])
swirs = pd.DataFrame("s3://usgs-landsat"+swirs['Swirs'])

In [ ]:
#make a new table with S3 links for objects
df = pd.concat([ids, dates, greens, swirs], axis = 1, sort=False)
df

In [ ]:
#Add Date/Time Constructor to Date Column of DF
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#Make a seasonality List to show only products where month acquired is May to September
df = df[df['Date'].dt.month.between(5, 9)]
df = df.reset_index(drop=True)
df

## Purpose: Extracting an area of interest from a Landsat scene and visualizing change over time by using Xrarry and Holoviews.

In [ ]:
# from dask_kubernetes import KubeCluster
# from dask.distributed import Client
# from dask.distributed import wait, progress
import os
import geopandas as gpd
import folium
from folium import Marker
import cartopy.crs as ccrs
import xarray as xr
import hvplot.xarray
import holoviews as hv
import rasterio as rio
from rasterio.session import AWSSession
import boto3
hv.extension('bokeh', width=90)

In [ ]:
#Show scene extent against AOI. We will eventuly slice the array by the AOI to process only the data we need
gf = features[1]['geometry']
geoms = gpd.read_file('Waubay_AOI.geojson')
m = folium.Map([46.0732, -97.7783], zoom_start=7, width=900, height=350, tiles='OpenStreetMap')
folium.GeoJson(gf).add_to(m)
folium.GeoJson(geoms).add_to(m)
m.add_child(Marker(location=[45.3676, -97.4048], popup='Waubay, South Dakota AOI', icon = folium.Icon(color = 'red')))
folium.LatLngPopup().add_to(m)
m

# No Kubernetes on the mini-pangeo - just use client

In [ ]:
#This allows us to use Dask with the requester pays bucket
os.environ["AWS_REQUEST_PAYER"] = "requester" 
#cluster = KubeCluster(env={'AWS_REQUEST_PAYER': 'requester'})
# cluster
#Click Manual Scaling
#Add 10 to the Workers Field
#Click Scale
#Click the Dashboard Link and open in a new window to view compute

In [ ]:
#client = Client()

In [ ]:
#client

In [ ]:
#client = Client(cluster)

In [ ]:
aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
#Extract corners from AOI to use to subset array
crs = ccrs.epsg(32614)
bounds = geoms.geometry.bounds
bounds.describe()

In [ ]:
lower_left_corner_lat_lon = bounds.minx.min(), bounds.miny.min()
upper_right_corner_lat_lon = bounds.maxx.max(), bounds.maxy.max()

print(lower_left_corner_lat_lon, upper_right_corner_lat_lon)

In [ ]:
ll_corner = crs.transform_point(*lower_left_corner_lat_lon, ccrs.PlateCarree())
ur_corner = crs.transform_point(*upper_right_corner_lat_lon, ccrs.PlateCarree())

print(ll_corner, ur_corner)

In [ ]:
#Function used to create array from data stored in dataframe table created above
def create_dataset(row, bands = ['Swirs', 'Green'], chunks = {'band': 1, 'x':2048, 'y':2049}):
    datasets = []
    with rio.Env(aws_session):
        for band in bands:
            url = row[band]
            #da = xr.open_rasterio(url, chunks = chunks)
            da = xr.open_rasterio(url)
            daSub = da.sel(x=slice(ll_corner[0], ur_corner[0]), y=slice(ur_corner[1], ll_corner[1]))
            daSub = daSub.squeeze().drop(labels='band')
            DS = daSub.to_dataset(name = band)
            datasets.append(DS)
        DS = xr.merge(datasets)
        return DS

In [ ]:
#load data
datasets = []
for i,row in df.iterrows():
    try:
        print('loading....', row.Date)
        ds = create_dataset(row)
        datasets.append(ds)
    except Exception as e:
        print('Error loading, skipping')
        print(e)

In [ ]:
DS = xr.concat(datasets, dim= pd.DatetimeIndex(df.Date.tolist(), name='time'))
print('Dataset Size (Gb): ', DS.nbytes/1e9)
DS

In [ ]:
#Calculate NDWI
MNDWI = (DS['Green'] - DS['Swirs']) / (DS['Green'] + DS['Swirs'])

In [ ]:
%%time
import pickle

with open('waubay_ndwi.pickle', 'wb') as handle:
    pickle.dump(MNDWI, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
img = MNDWI.hvplot('x', 'y', groupby='time', dynamic=True, rasterize=True, width=700, height=500, cmap='BrBG')
#use the time slider to step through time.

In [ ]:
img

In [ ]:
#! pip install --user cartopy

In [ ]:
#! pip install --user 

In [ ]:
#! pip install --user dask_kubernetes

In [ ]:
#! pip install --user pyepsg

In [ ]:
# import pickle

# with open('drb150c_xarray.pickle', 'rb') as handle:
#     da = pickle.load(handle)

In [ ]:
MNDWI.to_netcdf('water.nc')

In [ ]:
#! echo .nc >> .gitignore

In [ ]:
! cat .gitignore

In [ ]:
! ls -lh